In [31]:
import numpy as np
import os,glob
import scipy.ndimage as sni
import sys
import re
import itertools
from shutil import copytree
from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import make_lupton_rgb
import aplpy
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Arrow
from photutils import SkyCircularAnnulus
from photutils import SkyCircularAperture
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import SkyEllipticalAperture
from photutils import SkyEllipticalAnnulus
from photutils import EllipticalAperture
from photutils import EllipticalAnnulus
from photutils import aperture_photometry
import numpy.ma as ma
import math
import shutil
import pandas as pd

In [32]:
Dir='/home/heh15/workingspace/Arp240/NGC5257/ratio/'
scriptDir=Dir+'script/measure_1213/'
workDir=Dir+'mask/'
imageDir=Dir+'image/'
logDir=Dir+'log/'
ratioDir=Dir+'1213/contsub/'
maskDir=Dir+'mask/'

In [33]:
# function
def fits_import(fitsimage):
    hdr = fits.open(fitsimage)[0].header
    wcs = WCS(hdr).celestial
    data=fits.open(fitsimage)[0].data[0][0]
    position=SkyCoord(dec=50.4167*u.arcmin,ra=204.9706*u.degree,frame='icrs')
    size=u.Quantity((54,42),u.arcsec)
    cut=Cutout2D(data=data,position=position,size=size,wcs=wcs)
    data_cut=cut.data
    wcs_cut=cut.wcs
    data_masked=np.ma.masked_invalid(data_cut)
    return wcs_cut, data_masked

def masked_convert(data_masked,region_masked):
    data_mask=data_masked.mask
    region_mask=np.ma.make_mask(region_masked==0)
    region_mask=np.ma.mask_or(data_mask,region_mask)
    data_region=np.ma.masked_where(region_mask,data_masked)
    return data_region

def flux_mask_get(data_region,rms,chans,chan_width):
    flux=np.ma.sum(data_region)/beam_area_pix
    chans_tmp=chans+np.zeros((np.shape(data_region)[0],np.shape(data_region)[1]))
    error=np.sqrt(chans_tmp)*rms*chan_width/math.sqrt(beam_area_pix)
    error_masked=np.ma.masked_where(data_region.mask,error)
    uncertainty=math.sqrt(np.ma.sum(np.power(error_masked,2)))
    return flux, uncertainty

def ratio_cal(flux1,flux2,uncertainty1,uncertainty2):
    ratio=flux1/flux2
    uncertainty=math.sqrt((uncertainty1/flux1)**2+(uncertainty2/flux2)**2+2*0.05**2)*ratio
    return ratio, uncertainty

In [34]:
beam_area=2.186*1.896*1.1331
beam_area_pix=beam_area/0.09
regions=['center','hinge','south','rest','all']
values=['12CO10 flux','12CO10 uncertainty','13CO10 flux', '13CO10 uncertainty', '12CO21 flux',
        '12CO21 uncertainty','12/13','12/13 uncertainty','21/10','21/10 uncertainty']
CO13=['13CO10 flux', '13CO10 uncertainty']
ratio_table=pd.DataFrame(index=regions,columns=values)

values2=['12CO10 intensity','12CO10 uncertainty','13CO10 intensity','13CO10 uncertainty',
         '12CO21 intensity','12CO21 uncertainty']
intensities=pd.DataFrame(columns=values2,index=regions)

## 12CO 1-0

In [35]:
fitsimage=imageDir+'12CO10/NGC5257_12CO10_combine_contsub_smooth_13COcut_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [36]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['center']=flux
ratio_table['12CO10 uncertainty']['center']=uncertainty

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['hinge']=flux
ratio_table['12CO10 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['south']=flux
ratio_table['12CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['rest']=flux
ratio_table['12CO10 uncertainty']['rest']=uncertainty

In [37]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['center']=intensity
intensities['12CO10 uncertainty']['center']=intensity*ratio_table['12CO10 uncertainty']['center']\
                                  /ratio_table['12CO10 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['hinge']=intensity
intensities['12CO10 uncertainty']['hinge']=intensity*ratio_table['12CO10 uncertainty']['hinge']\
                                  /ratio_table['12CO10 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['south']=intensity
intensities['12CO10 uncertainty']['south']=intensity*ratio_table['12CO10 uncertainty']['south']\
                                  /ratio_table['12CO10 flux']['south']

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['rest']=intensity
intensities['12CO10 uncertainty']['rest']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

In [38]:
print(intensities)

       12CO10 intensity 12CO10 uncertainty 13CO10 intensity  \
center          5.20973          0.0549751              NaN   
hinge           1.57375          0.0345666              NaN   
south          0.583567          0.0667293              NaN   
rest            1.00937          0.0136897              NaN   
all                 NaN                NaN              NaN   

       13CO10 uncertainty 12CO21 intensity 12CO21 uncertainty  
center                NaN              NaN                NaN  
hinge                 NaN              NaN                NaN  
south                 NaN              NaN                NaN  
rest                  NaN              NaN                NaN  
all                   NaN              NaN                NaN  


## 13CO 1-0 

In [39]:
fitsimage=imageDir+'13CO10/NGC5257_13CO10_12m_contsub_smooth_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

Import the count channel map for 13CO1-0

In [40]:
fitsimage=imageDir+'13CO10/NGC5257_13CO10_12m_smooth_nchan.fits'
wcs=fits_import(fitsimage)[0]
chans_masked=fits_import(fitsimage)[1]
chans=chans_masked.data

In [41]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['center']=flux
ratio_table['13CO10 uncertainty']['center']=uncertainty

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['hinge']=flux
ratio_table['13CO10 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['south']=flux
ratio_table['13CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['rest']=flux
ratio_table['13CO10 uncertainty']['rest']=uncertainty

Calculate the intensity of the 13CO 1-0

In [42]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['center']=intensity
intensities['13CO10 uncertainty']['center']=intensity*ratio_table['13CO10 uncertainty']['center']\
                                  /ratio_table['13CO10 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['hinge']=intensity
intensities['13CO10 uncertainty']['hinge']=intensity*ratio_table['13CO10 uncertainty']['center']\
                                  /ratio_table['13CO10 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['south']=intensity
intensities['13CO10 uncertainty']['south']=intensity*ratio_table['13CO10 uncertainty']['south']\
                                  /ratio_table['13CO10 flux']['south']

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['rest']=intensity
intensities['13CO10 uncertainty']['rest']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

In [43]:
print(intensities)

       12CO10 intensity 12CO10 uncertainty 13CO10 intensity  \
center          5.20973          0.0549751         0.443934   
hinge           1.57375          0.0345666          0.20123   
south          0.583567          0.0667293         0.179378   
rest            1.00937          0.0136897         0.190574   
all                 NaN                NaN              NaN   

       13CO10 uncertainty 12CO21 intensity 12CO21 uncertainty  
center          0.0181139              NaN                NaN  
hinge            0.024655              NaN                NaN  
south           0.0435248              NaN                NaN  
rest            0.0137238              NaN                NaN  
all                   NaN              NaN                NaN  


## 12CO21 

In [44]:
fitsimage=imageDir+'12CO21/NGC5257_12CO21_combine_contsub_uvtaper_smooth_regrid_13COcut_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [45]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['center']=flux
ratio_table['12CO21 uncertainty']['center']=uncertainty

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['hinge']=flux
ratio_table['12CO21 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['south']=flux
ratio_table['12CO21 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['rest']=flux
ratio_table['12CO21 uncertainty']['rest']=uncertainty

In [46]:
# center
maskimage=maskDir+'center_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['center']=intensity
intensities['12CO21 uncertainty']['center']=intensity*ratio_table['12CO21 uncertainty']['center']\
                                  /ratio_table['12CO21 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['hinge']=intensity
intensities['12CO21 uncertainty']['hinge']=intensity*ratio_table['12CO21 uncertainty']['center']\
                                  /ratio_table['12CO21 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['south']=intensity
intensities['12CO21 uncertainty']['south']=intensity*ratio_table['12CO21 uncertainty']['south']\
                                  /ratio_table['12CO21 flux']['south']

# rest
maskimage=maskDir+'rest_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['rest']=intensity
intensities['12CO21 uncertainty']['rest']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

In [47]:
ratio_table['12/13']=ratio_table['12CO10 flux']/ratio_table['13CO10 flux']
fraction=((ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2+
                                  (ratio_table['13CO10 uncertainty']/ratio_table['13CO10 flux'])**2)**0.5
ratio_table['12/13 uncertainty']=ratio_table['12/13']*fraction

In [48]:
ratio_table['21/10']=ratio_table['12CO21 flux']/ratio_table['12CO10 flux']*115.27**2/230.54**2
fraction=((ratio_table['12CO21 uncertainty']/ratio_table['12CO21 flux'])**2+
                                  (ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2)**0.5
ratio_table['21/10 uncertainty']=ratio_table['21/10']*fraction
print(ratio_table)

       12CO10 flux 12CO10 uncertainty 13CO10 flux 13CO10 uncertainty  \
center     22.0644           0.232833     1.66748          0.0680382   
hinge       16.859             0.3703    0.555318          0.0344174   
south      1.67752            0.19182   0.0412512          0.0100093   
rest       68.9403           0.935011     3.88955           0.280098   
all            NaN                NaN         NaN                NaN   

       12CO21 flux 12CO21 uncertainty    12/13 12/13 uncertainty     21/10  \
center     72.8302           0.451113  13.2322          0.557679  0.825199   
hinge      55.9036           0.716814  30.3592           1.99626  0.828986   
south      9.33013           0.408253  40.6659           10.9081   1.39046   
rest       213.333            1.78186  17.7245           1.29883  0.773615   
all            NaN                NaN      NaN               NaN       NaN   

       21/10 uncertainty  
center         0.0100971  
hinge          0.0210838  
south           0

In [49]:
print(intensities)

       12CO10 intensity 12CO10 uncertainty 13CO10 intensity  \
center          5.20973          0.0549751         0.443934   
hinge           1.57375          0.0345666          0.20123   
south          0.583567          0.0667293         0.179378   
rest            1.00937          0.0136897         0.190574   
all                 NaN                NaN              NaN   

       13CO10 uncertainty 12CO21 intensity 12CO21 uncertainty  
center          0.0181139          17.1962           0.106514  
hinge            0.024655          5.22781          0.0421857  
south           0.0435248          2.68982           0.117697  
rest            0.0137238          3.22853          0.0269662  
all                   NaN              NaN                NaN  
